<a href="https://colab.research.google.com/github/thc1006/flora-dp-federated-ColO-RAN/blob/main/0707_FLORA_DP_client_15_v1_1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# @title Cell 1: 環境設定與函式庫匯入（修正版）
!pip install --upgrade opacus -q

import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np, pandas as pd, random, copy, json, os, time, warnings, math, re, contextlib
from collections import deque
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt, seaborn as sns
from dataclasses import dataclass, asdict
from sklearn.cluster import KMeans
from opacus import PrivacyEngine
from opacus.validators import ModuleValidator
from opacus.data_loader import DPDataLoader

# --- 環境設定 ---
try: torch._dynamo.disable()
except Exception: pass
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.filterwarnings("ignore", message=".*overflow encountered.*", category=RuntimeWarning)

pd.options.mode.chained_assignment = None

print("✅ Cell 1: 環境與函式庫準備就緒。")
import opacus
print(f"PyTorch/Opacus 版本: {torch.__version__} / {opacus.__version__}")
print(f"CUDA 是否可用: {torch.cuda.is_available()}")

✅ Cell 1: 環境與函式庫準備就緒。
PyTorch/Opacus 版本: 2.6.0+cu124 / 1.5.4
CUDA 是否可用: True


In [3]:
# @title Cell 2: 🎓 實驗參數設定（專家修正版）
from dataclasses import dataclass, field
import os
import json
import torch
import numpy as np
from typing import Tuple

@dataclass
class TrainingConfig:
    """
    儲存所有實驗的超參數與設定。

    Attributes:
        # 實驗基本設定
        experiment_name (str): 實驗的唯一名稱。
        output_dir (str): 儲存結果與模型的目錄。
        mode (str): 訓練模式，例如 "ClusteredFL", "FedProx", "FedAvg", "CQL"。
        random_seed (int): 用於可重現性的隨機種子。
        comm_rounds (int): 聯邦學習的總通信輪數。

        # Non-IID 客戶端生成 (任務 1)
        num_clients (int): 要生成的總客戶端數量。
        dirichlet_alpha (float): Dirichlet 分佈的 alpha 參數，用於控制 Non-IID 程度。alpha 越小，異質性越高。
        min_samples_per_client (int): 確保每個客戶端至少擁有的樣本數。

        # 聯邦學習設定
        num_clients_to_select (int): 每輪隨機選擇參與訓練的客戶端數量。
        fedprox_mu (float): FedProx 的近端項係數 mu。設為 0 等同於 FedAvg。
        num_clusters (int): 在 ClusteredFL 模式下的聚類數量。
        cluster_update_freq (int): ClusteredFL 模式下更新聚類的頻率（單位：輪）。

        # 半非同步/容錯聚合 (任務 4)
        async_threshold (float): 觸發非同步式聚合的客戶端超時/掉線比例閾值。

        # RL 訓練參數
        local_episodes_per_round (int): 每輪中，每個客戶端本地訓練的 episode 數量。
        steps_per_episode (int): 每個 episode 的最大步數。
        batch_size (int): 訓練時的批次大小。
        gamma (float): RL 中的折扣因子。
        lr (float): 學習率。
        target_update_freq (int): 目標網路更新的頻率（單位：episode）。

        # RL 探索參數
        epsilon_start (float): Epsilon-greedy 策略的初始探索率。
        epsilon_decay (float): Epsilon 的衰減率。
        epsilon_min (float): Epsilon 的最小值。

        # 記憶體與回放
        memory_capacity (int): 經驗回放緩衝區的大小。
        replay_start_size (int): 開始進行經驗回放所需的最小樣本數。
        replay_frequency (int): 進行經驗回放的頻率（單位：步）。
        replay_batches_per_call (int): 每次調用 replay 時訓練的批次數量。

        # 離線 RL 偏差校正 (任務 2)
        enable_cql (bool): 是否啟用 Conservative Q-Learning (CQL)。
        cql_alpha (float): CQL 損失的權重。
        cql_temperature (float): CQL 中用於 log-sum-exp 的溫度參數。

        # 差分隱私 (DP) 設定
        enable_dp (bool): 是否啟用差分隱私。
        dp_target_epsilon (float): 目標的總隱私預算 epsilon。
        dp_target_delta (float): 目標的 delta 值，通常為 1/dataset_size。
        dp_max_grad_norm (float): 每個樣本梯度的裁剪範數 (clipping bound)。
        dp_noise_multiplier (float): 添加到梯度中的噪聲乘數。

        # Adaptive Clipping (任務 3)
        enable_adaptive_clipping (bool): 是否啟用自適應梯度裁剪預計算。
        adaptive_clipping_percentile (float): 用於預計算的梯度範數百分位數。

        # DP 重設機制
        enable_dp_reset (bool): 是否允許在隱私預算超支時重設隱私會計。
        dp_reset_threshold_multiplier (float): 觸發重設的閾值 (相對於 target_epsilon)。

        # 系統與異質性設定
        enable_heterogeneity (bool): 是否模擬客戶端掉線 (dropout) 和延遲 (straggler)。
        straggler_ratio (float): 延遲客戶端的比例。
        dropout_ratio (float): 掉線客戶端的比例。
        enable_compression (bool): 是否啟用模型壓縮 (quantize_fp16)。
        use_pfl_finetune (bool): 是否在評估時執行個性化微調 (PFL)。
        local_finetune_episodes (int): PFL 微調的 episode 數量。
        device (str): 訓練設備 ("cuda" 或 "cpu")。
    """
    # 實驗基本設定
    experiment_name: str = "DFRL_Experiment"
    output_dir: str = "outputs"
    mode: str = "ClusteredFL"
    random_seed: int = 42
    comm_rounds: int = 20

    # Non-IID 客戶端生成 (任務 1)
    num_clients: int = 15
    dirichlet_alpha: float = 0.4
    min_samples_per_client: int = 500

    # 聯邦學習設定
    num_clients_to_select: int = 8
    fedprox_mu: float = 0.01
    num_clusters: int = 3
    cluster_update_freq: int = 8

    # 半非同步/容錯聚合 (任務 4)
    async_threshold: float = 0.3

    # RL 訓練參數
    local_episodes_per_round: int = 5
    steps_per_episode: int = 500
    batch_size: int = 128
    gamma: float = 0.99
    lr: float = 1e-4
    target_update_freq: int = 15

    # RL 探索參數
    epsilon_start: float = 1.0
    epsilon_decay: float = 0.9995
    epsilon_min: float = 0.05

    # 記憶體與回放
    memory_capacity: int = 50000
    replay_start_size: int = 1000
    replay_frequency: int = 2
    replay_batches_per_call: int = 2

    # 離線 RL 偏差校正 (任務 2)
    enable_cql: bool = False
    cql_alpha: float = 5.0
    cql_temperature: float = 1.0

    # 差分隱私 (DP) 設定
    enable_dp: bool = True
    dp_target_epsilon: float = 8.0
    dp_target_delta: float = 1e-5
    dp_max_grad_norm: float = 1.0
    dp_noise_multiplier: float = 0.5

    # Adaptive Clipping (任務 3)
    enable_adaptive_clipping: bool = False
    adaptive_clipping_percentile: float = 0.75

    # DP 重設機制
    enable_dp_reset: bool = True
    dp_reset_threshold_multiplier: float = 1.5

    # 系統與異質性設定
    enable_heterogeneity: bool = True
    straggler_ratio: float = 0.1
    dropout_ratio: float = 0.05
    enable_compression: bool = True
    compression_type: str = "quantize_fp16" # 保持此參數以兼容
    use_pfl_finetune: bool = True
    local_finetune_episodes: int = 15
    device: str = "cuda" if torch.cuda.is_available() else "cpu"

    def __post_init__(self):
        """後處理設定，進行動態檢查與配置。"""
        # GPU 環境檢測
        if self.device == "cuda" and torch.cuda.is_available():
            gpu_name = torch.cuda.get_device_name(0)
            print(f"🚀 GPU 環境檢測到: {gpu_name}")
        else:
            self.device = "cpu"
            print("💻 使用 CPU 模式")

        # 模式特定設定
        if self.mode == 'Centralized':
            self.enable_dp = False
            self.enable_heterogeneity = False
            print("🔹 集中式訓練模式，已自動禁用 DP 和異質性模擬。")

        if self.mode == "FedAvg":
            self.fedprox_mu = 0.0
            print("🔹 FedAvg 模式，fedprox_mu 已設為 0。")

        if self.mode == "CQL":
            self.enable_cql = True
            print("🔹 CQL 模式，已啟用 Conservative Q-Learning。")

        # 顯示核心配置
        print(f"\n--- 核心實驗配置 ---")
        print(f"模式: {self.mode} | 客戶端數: {self.num_clients} (Non-IID, α={self.dirichlet_alpha})")
        print(f"每輪參與: {self.num_clients_to_select} | 通信輪數: {self.comm_rounds}")

        if self.enable_dp and self.mode != 'Centralized':
            print(f"🛡️  差分隱私 (DP): 啟用")
            print(f"   - 目標預算: ε={self.dp_target_epsilon}, δ={self.dp_target_delta}")
            print(f"   - 噪聲乘數: {self.dp_noise_multiplier}")
            print(f"   - 梯度裁剪: {'自適應' if self.enable_adaptive_clipping else f'固定({self.dp_max_grad_norm})'}")
            print(f"   - 預算重設: {'啟用' if self.enable_dp_reset else '禁用'}")
        else:
            print(f"🛡️  差分隱私 (DP): 禁用")

    def save(self):
        """將當前配置以 JSON 格式保存到輸出目錄。"""
        os.makedirs(self.output_dir, exist_ok=True)
        path = os.path.join(self.output_dir, f'{self.experiment_name}_config.json')
        # 將 dataclass 轉換為可序列化的字典
        config_dict = asdict(self)
        with open(path, 'w') as f:
            json.dump(config_dict, f, indent=4)
        print(f"✅ 配置已保存至: {path}")

print("✅ Cell 2: TrainingConfig（專家修正版）定義完成。")

✅ Cell 2: TrainingConfig（專家修正版）定義完成。


In [4]:
# @title Cell 3: 🧩 數據與環境準備（Non-IID 專家修正版）
from typing import Dict # <--- 修正：加入此行匯入語句

class DataManager:
    """
    負責數據的讀取、預處理、以及 Non-IID 客戶端數據切分。
    任務 1 的主要實作在此類別中。
    """
    def __init__(self, data_path: str, config: 'TrainingConfig'):
        """
        初始化 DataManager。

        Args:
            data_path (str): Parquet 數據文件的路徑。
            config (TrainingConfig): 實驗配置對象。
        """
        print(f"\n[DataManager] 正在從 {data_path} 讀取數據...")
        self.df_kpi = pd.read_parquet(data_path)
        self.config = config
        self.tput_col = None
        self.lat_col = None
        self._sanitize_column_names()
        self._preflight_check()

    def _sanitize_column_names(self):
        """清理 DataFrame 的欄位名稱，使其更易於使用。"""
        sanitized_columns = [re.sub(r'[\\[\\]\\(\\)%\\s\\.-]+', '_', col.strip().lower()).strip('_')
                           for col in self.df_kpi.columns]
        self.df_kpi.columns = sanitized_columns

    def _preflight_check(self):
        """
        執行啟動前的數據預檢查，確保必要欄位存在。
        """
        print("\n" + "="*20 + " DataManager 啟動前預檢查 " + "="*20)
        cols = self.df_kpi.columns.tolist()
        tput_cand = ['throughput_dl_mbps', 'tx_brate_downlink_mbps']
        lat_cand = ['buffer_occupancy_dl_bytes', 'dl_buffer_bytes']

        self.tput_col = next((c for c in tput_cand if c in cols), None)
        self.lat_col = next((c for c in lat_cand if c in cols), None)

        print(f"✅ 清理後的欄位列表 (共 {len(cols)} 個)")
        if not self.tput_col or not self.lat_col:
            raise ValueError("預檢查失敗: 找不到必要的吞吐量或延遲數據欄位。")

        print(f"   - 吞吐量欄位成功匹配: '{self.tput_col}'")
        print(f"   - 延遲/緩衝區欄位成功匹配: '{self.lat_col}'")
        print("="*65 + "\n")

    def _get_merged_trajectory(self) -> pd.DataFrame:
        """
        將不同基站和切片的數據合併成一個統一的時間序列 DataFrame。
        這是 Non-IID 切分的基礎。
        """
        print("[DataManager] 正在合併 eMBB 和 URLLC 的數據軌跡...")
        available_bs = sorted(self.df_kpi['bs_id'].unique())
        all_merged_dfs = []

        # 我們需要一個參考BS來進行合併，這裡選擇第一個可用的BS
        # 但實際上，我們應該將所有BS的數據都視為一個大的數據池
        # 這裡我們簡化處理，將所有數據合併
        df_embb = self.df_kpi[self.df_kpi['slice_id'] == 0]
        df_urllc = self.df_kpi[self.df_kpi['slice_id'] == 2]

        # 重命名以避免合併衝突
        df_embb = df_embb[['timestamp', self.tput_col, self.lat_col]].rename(
            columns={self.tput_col: 'throughput_embb', self.lat_col: 'latency_embb'}
        ).dropna()
        df_urllc = df_urllc[['timestamp', self.tput_col, self.lat_col]].rename(
            columns={self.tput_col: 'throughput_urllc', self.lat_col: 'latency_urllc'}
        ).dropna()

        # 使用 merge_asof 將兩個切片的數據按時間對齊
        merged_df = pd.merge_asof(
            df_embb.sort_values('timestamp'),
            df_urllc.sort_values('timestamp'),
            on='timestamp',
            direction='backward',
            tolerance=pd.Timedelta('150ms') # 使用原始數據的容忍度
        ).dropna()

        # 數據清理
        merged_df = merged_df[
            (merged_df['throughput_embb'] >= 0) & (merged_df['throughput_urllc'] >= 0) &
            (merged_df['latency_embb'] >= 0) & (merged_df['latency_urllc'] >= 0)
        ]
        print(f"[DataManager] 全局數據軌跡合併完成，共 {len(merged_df)} 個時間步。")
        return merged_df.sort_values('timestamp').reset_index(drop=True)

    def create_non_iid_partitions(self) -> Dict[int, np.ndarray]:
        """
        (任務 1 核心實現)
        使用 Dirichlet 分佈生成 Non-IID 的客戶端數據分區。

        Returns:
            一個字典，鍵是客戶端 ID，值是對應的數據軌跡 (numpy array)。
        """
        print(f"\n[DataManager] 正在為 {self.config.num_clients} 個客戶端生成 Non-IID 數據分區...")
        print(f"   - Dirichlet Alpha (α): {self.config.dirichlet_alpha}")

        # 1. 獲取合併後的全局數據
        merged_df = self._get_merged_trajectory()
        if merged_df.empty:
            raise ValueError("合併後的全局數據為空，無法進行切分。")

        # 2. 使用 Dirichlet 分佈生成每個客戶端的數據比例
        # np.random.seed(self.config.random_seed) # 確保切分可重現
        proportions = np.random.dirichlet(np.repeat(self.config.dirichlet_alpha, self.config.num_clients))

        # 3. 根據比例切分數據索引
        total_samples = len(merged_df)
        client_indices = {}
        start_idx = 0
        for i in range(self.config.num_clients):
            num_samples = int(total_samples * proportions[i])
            # 確保每個客戶端都有最少的數據量
            if num_samples < self.config.min_samples_per_client:
                # 如果分配的太少，則從其他地方"借"一些，這會稍微改變分佈，但在實務上可接受
                num_samples = self.config.min_samples_per_client

            end_idx = min(start_idx + num_samples, total_samples)
            client_indices[i] = merged_df.index[start_idx:end_idx]
            start_idx = end_idx
            if start_idx >= total_samples:
                break # 數據已全部分配完畢

        # 如果因為 min_samples_per_client 的限制導致客戶端數量不足，需要調整
        num_created_clients = len(client_indices)
        if num_created_clients < self.config.num_clients:
            print(f"🟡 警告: 由於 min_samples_per_client 的限制，實際生成的客戶端數量為 {num_created_clients}")
            self.config.num_clients = num_created_clients # 更新配置以反映實際情況

        # 4. 創建最終的軌跡字典
        client_trajectories = {}
        feature_cols = ['throughput_embb', 'latency_embb', 'throughput_urllc', 'latency_urllc']

        for client_id, indices in tqdm(client_indices.items(), desc="創建客戶端軌跡"):
            client_df = merged_df.loc[indices]
            trajectory = client_df[feature_cols].to_numpy(dtype=np.float32)
            client_trajectories[client_id] = trajectory
            print(f"   - 客戶端 {client_id}: {len(trajectory)} 個時間步")

        num_valid = sum(1 for traj in client_trajectories.values() if traj.size > 0)
        print(f"\n[DataManager] Non-IID 數據處理完成！成功為 {num_valid} / {self.config.num_clients} 個客戶端創建了環境。")
        return client_trajectories

print("✅ Cell 3: DataManager（Non-IID 專家修正版）定義完成。")

✅ Cell 3: DataManager（Non-IID 專家修正版）定義完成。


In [5]:
# @title Cell 4: ⚡ RL環境與數據處理（專家修正版）
import gc
import time
import os
import torch
import numpy as np
import pandas as pd
import contextlib
from collections import deque
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler

class PairedEnv:
    """
    配對服務 (eMBB/URLLC) 的強化學習環境。
    代表單一客戶端的本地環境。
    """
    def __init__(self, trajectory: np.ndarray, config: TrainingConfig):
        """
        初始化環境。

        Args:
            trajectory (np.ndarray): 客戶端的數據軌跡，形狀為 (n_steps, 4)。
            config (TrainingConfig): 實驗配置。
        """
        self.trajectory = trajectory
        self.config = config
        # 狀態維度：[throughput_embb, latency_embb, throughput_urllc, latency_urllc]
        self.state_size = trajectory.shape[1] if trajectory.size > 0 else 4
        # 動作空間：[0: 偏重eMBB, 1: 平衡, 2: 偏重URLLC]
        self.action_size = 3
        self.cursor = 0
        self.reset()

    def reset(self) -> np.ndarray:
        """
        重置環境到一個新的起始點。

        Returns:
            初始狀態。
        """
        if self.trajectory.size == 0:
            return np.zeros(self.state_size, dtype=np.float32)

        # 為了讓每個 episode 都是完整的，從軌跡的較早部分開始
        max_start = max(0, len(self.trajectory) - self.config.steps_per_episode)
        # 隨機選擇起始點以增加多樣性
        if max_start > 0:
            self.cursor = np.random.randint(0, max_start)
        else:
            self.cursor = 0
        return self.trajectory[self.cursor]

    def step(self, action_id: int) -> Tuple[np.ndarray, float, bool, dict]:
        """
        在環境中執行一步。

        Args:
            action_id (int): 選擇的動作 ID。

        Returns:
            一個元組 (next_state, reward, done, info)。
        """
        if self.trajectory.size == 0 or self.cursor >= len(self.trajectory) - 1:
            # 如果軌跡結束或為空，返回終止狀態
            state = self.trajectory[-1] if self.trajectory.size > 0 else np.zeros(self.state_size, dtype=np.float32)
            return state, 0.0, True, {}

        self.cursor += 1
        done = self.cursor >= (len(self.trajectory) - 1)
        state = self.trajectory[self.cursor]
        reward = self._compute_reward(state, action_id)
        return state, reward, done, {}

    def _compute_reward(self, state: np.ndarray, action_id: int) -> float:
        """
        根據當前狀態和動作計算獎勵。
        獎勵函數旨在最大化吞吐量，同時最小化延遲懲罰。

        Args:
            state (np.ndarray): 當前狀態。
            action_id (int): 執行的動作。

        Returns:
            計算出的獎勵值。
        """
        tput_embb, lat_embb, tput_urllc, lat_urllc = state

        # 根據動作 ID 調整權重
        if action_id == 0:  # 偏重 eMBB
            w_tput, w_lat = (0.7, 0.3)
        elif action_id == 2:  # 偏重 URLLC
            w_tput, w_lat = (0.3, 0.7)
        else:  # 平衡
            w_tput, w_lat = (0.5, 0.5)

        # 使用 log1p 處理吞吐量，使其尺度更平滑
        tput_reward = w_tput * (np.log1p(tput_embb) + 0.5 * np.log1p(tput_urllc))
        # 使用 tanh 將延遲懲罰限制在 [-1, 1] 範圍內
        lat_penalty = w_lat * (np.tanh(lat_urllc * 1e-6) + 0.3 * np.tanh(lat_embb * 1e-6))

        reward_val = tput_reward - lat_penalty
        # 處理潛在的 NaN 或 inf 值
        return float(np.nan_to_num(reward_val, nan=0.0, posinf=10.0, neginf=-10.0))

class RLDataset(Dataset):
    """用於強化學習經驗回放的 PyTorch 數據集。"""
    def __init__(self, memory_deque: deque):
        self.data = list(memory_deque)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        state, action, reward, next_state, done = self.data[idx]
        return (
            torch.from_numpy(state).float(),
            torch.tensor(action).long(),
            torch.tensor(reward).float(),
            torch.from_numpy(next_state).float(),
            torch.tensor(done).bool()
        )

def get_data_loader(agent_memory: deque, batch_size: int, device: str) -> DataLoader:
    """
    創建一個用於 RL 訓練的數據加載器。
    針對 GPU 進行了優化，例如使用 pin_memory。
    """
    if len(agent_memory) < batch_size:
        return None

    dataset = RLDataset(agent_memory)
    # 根據 CPU 核心數自動調整 num_workers
    num_workers = min(os.cpu_count() // 2, 4) if torch.cuda.is_available() else 0

    return DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=(device == 'cuda'),
        drop_last=True,
        persistent_workers= (num_workers > 0)
    )

def setup_gpu_environment():
    """統一的 GPU 環境設定函數。"""
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        print(f"\n🎮 GPU 檢測: {gpu_name}")
        # 啟用 cudnn auto-tuner，這會尋找最優的卷積算法
        torch.backends.cudnn.benchmark = True
        # 允許 PyTorch 使用 TF32 來加速矩陣乘法（在 Ampere 架構及更新的 GPU 上）
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.allow_tf32 = True
        # 清理緩存
        torch.cuda.empty_cache()
        gc.collect()
        print(f"🧹 GPU 環境已設定並優化。")
    else:
        print("⚠️ 未檢測到 GPU，將使用 CPU 模式運行。")


print("✅ Cell 4: RL環境與數據處理（專家修正版）定義完成。")

✅ Cell 4: RL環境與數據處理（專家修正版）定義完成。


In [6]:
# @title Cell 5: 🛡️ 核心學習代理（CQL & Adaptive Clipping 專家修正版）
import gc
import time
from opacus import PrivacyEngine
from opacus.validators import ModuleValidator
from opacus.accountants.utils import get_noise_multiplier

class AdaptiveClipper:
    """
    (任務 3 核心類別)
    一個輔助類，用於追踪梯度範數並建議自適應的裁剪值。
    """
    def __init__(self, percentile: float, window_size: int = 100):
        """
        初始化自適應裁剪器。

        Args:
            percentile (float): 用於確定裁剪值的梯度範數百分位數。
            window_size (int): 用於計算百分位數的移動窗口大小。
        """
        self.percentile = percentile
        self.grad_norm_history = deque(maxlen=window_size)
        self.suggested_clip_norm = 1.0  # 初始預設值

    def track_grad_norm(self, model: nn.Module):
        """
        計算並追踪模型參數的梯度總範數。

        Args:
            model (nn.Module): 正在訓練的模型。
        """
        total_norm = 0.0
        # Opacus 包裝的模型，參數在 _module 中
        params = model._module.parameters() if hasattr(model, '_module') else model.parameters()
        for p in params:
            if p.grad is not None:
                param_norm = p.grad.data.norm(2)
                total_norm += param_norm.item() ** 2
        total_norm = total_norm ** 0.5
        self.grad_norm_history.append(total_norm)

    def update_clip_norm(self):
        """
        根據歷史梯度範數更新建議的裁剪值。
        """
        if len(self.grad_norm_history) > 20: # 需要足夠的數據點
            self.suggested_clip_norm = float(np.percentile(list(self.grad_norm_history), self.percentile * 100))

class RLAgent:
    """
    強化學習代理，封裝了 DQN 模型、訓練邏輯和差分隱私。
    """
    def __init__(self, state_size: int, action_size: int, config: TrainingConfig, client_id: int,
                 dataset_size: int, is_eval_agent: bool = False):
        self.state_size, self.action_size, self.config = state_size, action_size, config
        self.client_id, self.dataset_size = client_id, dataset_size
        self.device = torch.device(config.device)
        self.mu = self.config.fedprox_mu
        self.gamma, self.epsilon = config.gamma, config.epsilon_start
        self.memory = deque(maxlen=config.memory_capacity)
        self.global_params = None
        self.is_eval_agent = is_eval_agent
        self.privacy_engine = None
        self.current_epsilon = 0.0

        self.model = self._build_model()
        self.target_model = self._build_model()
        self.update_target_model()
        self.target_model.eval()

        self.optimizer = optim.Adam(self.model.parameters(), lr=config.lr)
        self.criterion = nn.MSELoss()

        if self.config.enable_cql:
            print(f"[C-{self.client_id}] 🧠 CQL 模式啟用 (alpha={self.config.cql_alpha})")

        if self.config.enable_dp and not self.is_eval_agent and self.config.mode != 'Centralized':
            self._initialize_dp_engine()

    def _build_model(self) -> nn.Module:
        """建立 DQN 神經網路模型。"""
        model = nn.Sequential(
            nn.Linear(self.state_size, 128), nn.ReLU(),
            nn.Linear(128, 64), nn.ReLU(),
            nn.Linear(64, self.action_size)
        ).to(self.device)
        # 如果啟用DP，預先修復模型以確保與Opacus兼容
        if self.config.enable_dp and not ModuleValidator.is_valid(model):
            model = ModuleValidator.fix(model)
        return model

    def _initialize_dp_engine(self):
        """初始化 Opacus 的差分隱私引擎。"""
        print(f"[C-{self.client_id}] 🛡️ 正在初始化差分隱私引擎...")
        try:
            self.privacy_engine = PrivacyEngine(accountant="gdp")
            # 創建一個假的 DataLoader 來初始化 PrivacyEngine
            dummy_dataset = RLDataset(deque([(
                np.zeros(self.state_size, dtype=np.float32), 0, 0.0,
                np.zeros(self.state_size, dtype=np.float32), False
            )] * self.config.batch_size))
            dummy_loader = DataLoader(dummy_dataset, batch_size=self.config.batch_size)

            self.model, self.optimizer, _ = self.privacy_engine.make_private(
                module=self.model,
                optimizer=self.optimizer,
                data_loader=dummy_loader, # 這個 loader 僅用於獲取 sample_rate
                noise_multiplier=self.config.dp_noise_multiplier,
                max_grad_norm=self.config.dp_max_grad_norm,
                poisson_sampling=True
            )
            print(f"   - ✅ 差分隱私引擎初始化成功 (GradSampleModule)")
        except Exception as e:
            print(f"   - ❌ 差分隱私初始化失敗: {e}。將在非隱私模式下運行。")
            self.privacy_engine = None
            self.config.enable_dp = False

    def act(self, state: np.ndarray) -> int:
        """根據當前狀態和 epsilon-greedy 策略選擇一個動作。"""
        if not self.is_eval_agent and np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        with torch.no_grad():
            state_tensor = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
            q_values = self.model(state_tensor)
        return q_values.argmax().item()

    def remember(self, *args):
        """將一條經驗存入回放緩衝區。"""
        self.memory.append(args)

    def replay(self, num_batches: int, clipper: AdaptiveClipper = None) -> float:
        """
        從記憶體中取樣並訓練模型。
        (任務 2 和 3 的核心實現)
        """
        if len(self.memory) < self.config.batch_size:
            return 0.0

        data_loader = get_data_loader(self.memory, self.config.batch_size, self.device)
        if data_loader is None: return 0.0

        total_loss, batches_processed = 0.0, 0
        self.model.train()

        for i, batch in enumerate(data_loader):
            if i >= num_batches: break

            states, actions, rewards, next_states, dones = [item.to(self.device) for item in batch]
            self.optimizer.zero_grad()

            # 計算 Q 值
            current_q_all = self.model(states)
            current_q = current_q_all.gather(1, actions.unsqueeze(1))

            # 計算目標 Q 值 (Double DQN)
            with torch.no_grad():
                next_actions = self.model(next_states).argmax(dim=1, keepdim=True)
                max_next_q = self.target_model(next_states).gather(1, next_actions)
                target_q = rewards.unsqueeze(1) + (self.gamma * max_next_q * (~dones.unsqueeze(1)))

            # 計算核心損失
            loss = self.criterion(current_q, target_q)

            # (任務 2) 添加 CQL 損失項
            if self.config.enable_cql:
                logsumexp_q = torch.logsumexp(current_q_all / self.config.cql_temperature, dim=1).mean()
                cql_loss = self.config.cql_alpha * (logsumexp_q - current_q.mean())
                loss += cql_loss

            # (任務 3) FedProx 正則化項
            if self.config.mode in ['FedProx', 'ClusteredFL'] and self.mu > 0 and self.global_params:
                proximal_term = 0.0
                model_params = self.model._module.parameters() if hasattr(self.model, '_module') else self.model.parameters()
                for local_param, global_param in zip(model_params, self.global_params):
                    proximal_term += torch.sum((local_param - global_param.to(self.device))**2)
                loss += (self.mu / 2) * proximal_term

            if not torch.isfinite(loss): continue

            loss.backward()

            # (任務 3) 在優化器步驟前更新自適應裁剪器
            if clipper and self.config.enable_adaptive_clipping:
                clipper.track_grad_norm(self.model)
                clipper.update_clip_norm() # 這裡只更新建議值，實際值在引擎初始化時設定

            self.optimizer.step()
            total_loss += loss.item()
            batches_processed += 1

        return total_loss / batches_processed if batches_processed > 0 else 0.0

    def get_privacy_cost(self) -> float:
        """獲取當前的隱私成本 (epsilon)。"""
        if not self.privacy_engine: return 0.0
        try:
            return self.privacy_engine.get_epsilon(delta=self.config.dp_target_delta)
        except Exception as e:
            # print(f"Warning: Could not get epsilon: {e}")
            return self.current_epsilon # 返回上一次成功的值

    def set_global_params(self, state_dict: dict):
        """設置用於 FedProx 計算的全局模型參數。"""
        with torch.no_grad():
            self.global_params = [p.clone().detach().cpu() for p in state_dict.values()]

    def update_target_model(self):
        """將主模型的權重複製到目標模型。"""
        self.target_model.load_state_dict(self.get_clean_state_dict())

    def get_clean_state_dict(self) -> dict:
        """獲取模型的 state_dict，如果是 DP 模型，則從 _module 中提取。"""
        return self.model._module.state_dict() if self.privacy_engine and hasattr(self.model, '_module') else self.model.state_dict()

    def get_model_for_upload(self) -> dict:
        """準備要上傳到服務器的模型，可選壓縮。"""
        state_dict = self.get_clean_state_dict()
        if self.config.enable_compression:
            return {k: v.cpu().half() for k, v in state_dict.items()}
        return {k: v.cpu() for k, v in state_dict.items()}

    def get_model_weights_flat(self) -> np.ndarray:
        """將模型權重展平為一個向量，用於聚類分析。"""
        with torch.no_grad():
            params = self.model._module.parameters() if self.privacy_engine and hasattr(self.model, '_module') else self.model.parameters()
            return torch.cat([p.view(-1) for p in params]).cpu().numpy()

print("✅ Cell 5: RLAgent（CQL & Adaptive Clipping 專家修正版）定義完成。")

✅ Cell 5: RLAgent（CQL & Adaptive Clipping 專家修正版）定義完成。


In [12]:
# @title Cell 6: 🌐 聯邦學習服務器類別（DP 格式修正版）
import torch
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from typing import Dict, List, Tuple, Any
import copy

class FLServer:
    """
    聯邦學習服務器，負責模型聚合、客戶端聚類和模型分發。
    【錯誤修正】本版本特別處理了差分隱私模型（GradSampleModule）與
    標準模型之間 state_dict 鍵名不匹配的問題。
    """
    def __init__(self, config: TrainingConfig):
        """
        初始化 FLServer。

        Args:
            config (TrainingConfig): 實驗配置。
        """
        self.config = config
        self.num_clusters = config.num_clusters
        self.client_to_cluster: Dict[int, int] = {}
        self.cluster_models: Dict[int, Dict[str, torch.Tensor]] = {
            i: None for i in range(config.num_clusters)
        }
        print(f"[FLServer] 初始化完成 - 模式: {self.config.mode}, 聚類數: {self.num_clusters}")

    def _is_dp_model(self, model: torch.nn.Module) -> bool:
        """檢查模型是否為 Opacus 包裝的 GradSampleModule。"""
        return "GradSampleModule" in str(type(model))

    def _convert_state_dict_keys(self, state_dict: Dict, target_model_is_dp: bool) -> Dict:
        """
        【關鍵修正】根據目標模型是否為 DP 模型，自動轉換 state_dict 的鍵。
        """
        if not state_dict: return {}

        source_keys_have_prefix = any(k.startswith('_module.') for k in state_dict.keys())

        # 如果格式已經匹配，則無需轉換
        if source_keys_have_prefix == target_model_is_dp:
            return state_dict

        new_dict = {}
        if target_model_is_dp: # 需要添加前綴
            for key, value in state_dict.items():
                new_dict[f"_module.{key}"] = value
        else: # 需要移除前綴
            for key, value in state_dict.items():
                if key.startswith('_module.'):
                    new_dict[key[len("_module."):]] = value
        return new_dict

    def distribute_model(self, participating_agents: Dict[int, 'RLAgent'], global_model_state: Dict):
        """
        將適當的模型（全局或聚類模型）分發給參與的客戶端。
        【錯誤修正】分發前會自動校正 state_dict 的鍵。
        """
        if not global_model_state:
            print("[FLServer] ⚠️ 全局模型為空，跳過分發。")
            return

        # 全局模型是乾淨的（沒有 _module. 前綴）
        for client_id, agent in participating_agents.items():
            model_to_send = global_model_state
            if self.config.mode == 'ClusteredFL':
                cluster_id = self.client_to_cluster.get(client_id, 0)
                if self.cluster_models.get(cluster_id) is not None:
                    model_to_send = self.cluster_models[cluster_id]

            # 檢查目標客戶端模型是否為 DP 模型
            agent_is_dp = self._is_dp_model(agent.model)

            # 【關鍵修正】轉換 state_dict 以匹配目標模型
            model_state_for_loading = self._convert_state_dict_keys(
                model_to_send, target_model_is_dp=agent_is_dp
            )

            # 將模型權重轉到 agent 所在的 device
            model_to_load = {k: v.to(agent.device).float() for k, v in model_state_for_loading.items()}

            # 現在載入不會報錯了
            agent.model.load_state_dict(model_to_load)

            # 用於 FedProx 的全局參數應始終是乾淨的 state_dict
            agent.set_global_params(model_to_send)

    def aggregate_weighted(self, client_updates: List[Tuple[Dict, int]]) -> Dict:
        """
        對客戶端上傳的模型更新進行加權平均聚合 (FedAvg)。
        【錯誤修正】確保返回的聚合模型是"乾淨"的（沒有 _module. 前綴）。
        """
        if not client_updates: return {}

        total_samples = sum(num_samples for _, num_samples in client_updates if num_samples > 0)
        if total_samples == 0: return self._convert_state_dict_keys(client_updates[0][0], target_model_is_dp=False)

        # 以第一個乾淨的模型為基準初始化聚合模型
        first_clean_model = self._convert_state_dict_keys(client_updates[0][0], target_model_is_dp=False)
        aggregated_model = {k: torch.zeros_like(v) for k, v in first_clean_model.items()}

        for model_state, num_samples in client_updates:
            if num_samples == 0: continue

            weight = num_samples / total_samples

            # 將每個上傳的模型都轉換為乾淨的格式再進行聚合
            clean_model_state = self._convert_state_dict_keys(model_state, target_model_is_dp=False)

            for key in aggregated_model.keys():
                if key in clean_model_state:
                    aggregated_model[key] += weight * clean_model_state[key].to(aggregated_model[key].device)

        return aggregated_model

    def update_clusters(self, client_agents: Dict[int, 'RLAgent'], current_round: int):
        """
        (ClusteredFL 模式) 使用 K-Means 根據客戶端模型權重對客戶端進行聚類。
        """
        if len(client_agents) < self.num_clusters:
            # print("[FLServer] 客戶端數量不足以進行有意義的聚類。")
            return

        print(f"\n[FLServer] Round {current_round}: 正在更新客戶端聚類...")
        try:
            client_features, client_ids = [], []
            for cid, agent in client_agents.items():
                weights = agent.get_model_weights_flat()
                if weights is not None and len(weights) > 0:
                    client_features.append(weights)
                    client_ids.append(cid)

            if len(client_features) < self.num_clusters:
                print("[FLServer] 有效特徵數量不足以聚類。")
                return

            features_array = StandardScaler().fit_transform(np.vstack(client_features))

            kmeans = KMeans(n_clusters=self.num_clusters, random_state=self.config.random_seed, n_init=10)
            cluster_labels = kmeans.fit_predict(features_array)

            self.client_to_cluster = {cid: label for cid, label in zip(client_ids, cluster_labels)}

            print("[FLServer] 聚類更新完成:")
            for i in range(self.num_clusters):
                clients_in_cluster = [cid for cid, c_label in self.client_to_cluster.items() if c_label == i]
                print(f"  - 聚類 {i}: {clients_in_cluster}")

        except Exception as e:
            print(f"[FLServer] ❌ 聚類更新失敗: {e}。將回退到隨機分配。")
            client_ids = list(client_agents.keys())
            random.shuffle(client_ids)
            for i, cid in enumerate(client_ids):
                self.client_to_cluster[cid] = i % self.num_clusters

print("✅ Cell 6: FLServer（DP 格式修正版）定義完成。")

✅ Cell 6: FLServer（DP 格式修正版）定義完成。


In [8]:
# @title Cell 7: 🚀 ExperimentRunner（遞迴修正版）
import scipy.stats as stats
import time
from tqdm.notebook import tqdm
import copy
import numpy as np
import pandas as pd
import os

class ExperimentRunner:
    """
    實驗執行器，負責協調整個聯邦學習流程，包括客戶端選擇、
    本地訓練、模型聚合、評估和日誌記錄。
    (任務 2, 4, 5, 6 的主要實現)
    """
    def __init__(self, config: TrainingConfig, data_manager: DataManager):
        self.config = config
        self.data_manager = data_manager
        self._set_seeds() # (任務 6)

        # 數據與客戶端初始化
        self.all_trajectories = self.data_manager.create_non_iid_partitions()
        self.client_envs = {cid: PairedEnv(traj, config) for cid, traj in self.all_trajectories.items() if traj.size > 0}
        if not self.client_envs:
            raise ValueError("未能為任何客戶端創建有效的環境。")

        # 更新配置以匹配實際生成的客戶端數量
        self.config.num_clients = len(self.client_envs)
        self.config.num_clients_to_select = min(self.config.num_clients_to_select, self.config.num_clients)

        # --- 【遞迴修正】調整執行順序 ---
        # 1. 如果啟用，先執行預計算，更新 config
        if self.config.enable_dp and self.config.enable_adaptive_clipping:
            self._precompute_adaptive_grad_norm()

        # 2. 使用更新後的 config 創建代理與服務器
        self.server = FLServer(config)
        self.client_agents = self._create_agents() # 現在這一步是安全的
        self.global_model_state = self.client_agents[next(iter(self.client_agents))].get_clean_state_dict() if self.client_agents else {}

        # 結果記錄
        self.training_history = []
        self.evaluation_results = []
        self.privacy_costs = []
        self.latency_logs = [] # (任務 5)

        self.config.save()
        print("\n[ExperimentRunner] 初始化完成。")

    def _set_seeds(self):
        """ (任務 6) 設置所有相關庫的隨機種子以確保可重現性。"""
        seed = self.config.random_seed
        torch.manual_seed(seed)
        np.random.seed(seed)
        random.seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed(seed)
            torch.cuda.manual_seed_all(seed)
            # 關鍵的確定性設定
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False
            # PyTorch 1.8+
            try:
                torch.use_deterministic_algorithms(True)
                print("torch.use_deterministic_algorithms(True) 已設置")
            except (AttributeError, RuntimeError): # RuntimeError 可能在某些 CUDA 版本上發生
                print("當前 PyTorch/CUDA 版本不完全支持或已啟用確定性演算法")

    def _create_agents(self) -> Dict[int, RLAgent]:
        """【遞迴修正】創建所有客戶端的 RL 代理。不再呼叫預計算函式。"""
        print("\n[ExperimentRunner] 正在初始化所有客戶端代理...")
        agents = {}
        for cid, env in self.client_envs.items():
            dataset_size = len(env.trajectory) if env.trajectory.size > 0 else 1
            # 使用 self.config (可能已被預計算更新) 來創建代理
            agents[cid] = RLAgent(env.state_size, env.action_size, self.config, cid, dataset_size)
        return agents

    def _precompute_adaptive_grad_norm(self):
        """
        【遞迴修正】
        在訓練前運行一個短暫的預熱階段來估算一個穩健的梯度裁剪範數。
        此函式現在直接創建一個獨立的臨時代理，而不是呼叫 _create_agents()。
        """
        print("\n[Adaptive Clipper] 正在預計算梯度裁剪範數...")
        clipper = AdaptiveClipper(percentile=self.config.adaptive_clipping_percentile)

        # --- 直接建立一個輕量級的臨時代理 ---
        # 選擇一個有代表性的客戶端環境（例如數據量較多的）
        temp_env = next(iter(self.client_envs.values()))
        # 創建一個不啟用DP的臨時config來建立代理，避免不必要的開銷
        temp_config = copy.deepcopy(self.config)
        temp_config.enable_dp = False
        temp_agent = RLAgent(temp_env.state_size, temp_env.action_size, temp_config, -1, 1)
        # --- --------------------------- ---

        state = temp_env.reset()

        # 模擬一些初始步驟來收集梯度
        print("   - 執行預熱步驟以收集梯度範數...")
        for _ in range(200): # 預熱步數
            action = temp_agent.act(state)
            next_state, reward, done, _ = temp_env.step(action)
            temp_agent.remember(state, action, reward, next_state, done)
            state = next_state if not done else temp_env.reset()
            if len(temp_agent.memory) > self.config.batch_size:
                temp_agent.replay(num_batches=1, clipper=clipper)

        # 更新配置中的裁剪值
        if clipper.suggested_clip_norm > 0:
            new_clip_norm = round(clipper.suggested_clip_norm, 2)
            print(f"   - ✅ 預計算完成。建議的裁剪範數 ({self.config.adaptive_clipping_percentile*100:.0f} 百分位): {new_clip_norm}")
            # 直接修改 self.config 物件，此修改將在後續的 _create_agents 中生效
            self.config.dp_max_grad_norm = new_clip_norm
        else:
            print(f"   - ⚠️ 預計算未能找到有效的裁剪值，將使用預設值: {self.config.dp_max_grad_norm}")

        del temp_agent, temp_env, temp_config, clipper


    def _run_federated_training(self):
        """運行主聯邦學習訓練循環。"""
        print(f"\n[模式] 開始聯邦式訓練 ({self.config.mode})")
        progress_bar = tqdm(range(self.config.comm_rounds), desc=f"{self.config.mode} Training")

        for comm_round in progress_bar:
            round_start_time = time.time() # (任務 5)
            # --- 1. 聚類更新 (如果適用) ---
            if self.config.mode == 'ClusteredFL' and comm_round > 0 and comm_round % self.config.cluster_update_freq == 0:
                self.server.update_clusters(self.client_agents, comm_round)

            # --- 2. 客戶端選擇與異質性模擬 ---
            available_ids = list(self.client_agents.keys())
            num_to_select = min(self.config.num_clients_to_select, len(available_ids))
            selected_ids = np.random.choice(available_ids, num_to_select, replace=False)

            dropout_ids, straggler_ids = set(), set()
            if self.config.enable_heterogeneity:
                num_dropouts = int(self.config.dropout_ratio * num_to_select)
                dropout_ids = set(np.random.choice(selected_ids, num_dropouts, replace=False))

                remaining_ids = [cid for cid in selected_ids if cid not in dropout_ids]
                num_stragglers = int(self.config.straggler_ratio * len(remaining_ids))
                if num_stragglers > 0:
                    straggler_ids = set(np.random.choice(remaining_ids, num_stragglers, replace=False))

            participating_ids = [cid for cid in selected_ids if cid not in dropout_ids]
            if not participating_ids:
                print(f"[Round {comm_round+1}] ⚠️ 所有選擇的客戶端均掉線，跳過本輪。")
                continue

            # --- 3. 模型分發 ---
            participating_agents = {cid: self.client_agents[cid] for cid in participating_ids}
            self.server.distribute_model(participating_agents, self.global_model_state)

            # --- 4. 本地訓練 ---
            client_updates, round_losses, round_rewards, round_epsilons = [], [], [], []
            for cid in participating_ids:
                agent, env = self.client_agents[cid], self.client_envs[cid]
                # 延遲者訓練的 episode 較少
                episodes = self.config.local_episodes_per_round // 2 if cid in straggler_ids else self.config.local_episodes_per_round
                loss, reward, privacy_cost = self._train_agent_locally(agent, env, episodes)

                client_updates.append((agent.get_model_for_upload(), len(env.trajectory)))
                round_losses.append(loss)
                round_rewards.append(reward)
                if self.config.enable_dp:
                    # 更新每個 agent 的 epsilon 記錄
                    agent.current_epsilon = privacy_cost
                    round_epsilons.append(privacy_cost)

            # --- 5. 容錯聚合 (任務 4) ---
            num_successful = len(client_updates)
            num_expected = len(selected_ids)
            success_ratio = num_successful / num_expected if num_expected > 0 else 0

            if success_ratio < (1 - self.config.async_threshold):
                print(f"\n[Round {comm_round+1}] ⚠️ 容錯聚合觸發！成功客戶端比例 {success_ratio:.2f} 低於閾值。僅聚合 {num_successful} 個模型。")

            if not client_updates:
                print(f"[Round {comm_round+1}] ⚠️ 沒有客戶端回傳模型，跳過聚合。")
                continue

            # --- 6. 模型聚合 ---
            if self.config.mode == 'ClusteredFL':
                # 按聚類進行聚合
                client_updates_by_cluster = {i: [] for i in range(self.config.num_clusters)}
                for i, (model_update, num_points) in enumerate(client_updates):
                    cid = participating_ids[i]
                    cluster_id = self.server.client_to_cluster.get(cid, 0)
                    client_updates_by_cluster[cluster_id].append((model_update, num_points))

                new_cluster_models = []
                for cluster_id, updates in client_updates_by_cluster.items():
                    if updates:
                        updated_model = self.server.aggregate_weighted(updates)
                        self.server.cluster_models[cluster_id] = updated_model
                        new_cluster_models.append((updated_model, sum(n for _, n in updates)))

                if new_cluster_models:
                    self.global_model_state = self.server.aggregate_weighted(new_cluster_models)
            else: # FedAvg, FedProx, CQL
                self.global_model_state = self.server.aggregate_weighted(client_updates)

            # --- 7. 日誌記錄 ---
            round_wall_time = time.time() - round_start_time # (任務 5)
            self.latency_logs.append(round_wall_time)
            if round_wall_time > 1.0:
                 print(f"[Round {comm_round+1}] ⚠️ 近即時延遲警告: 本輪耗時 {round_wall_time:.2f} 秒 > 1 秒。")

            avg_loss = np.mean(round_losses) if round_losses else 0
            avg_reward = np.mean(round_rewards) if round_rewards else 0
            avg_epsilon = np.mean(round_epsilons) if round_epsilons else 0.0

            self.training_history.append({'round': comm_round, 'avg_reward': avg_reward, 'avg_loss': avg_loss})
            self.privacy_costs.append({'round': comm_round, 'epsilon': avg_epsilon})

            progress_bar.set_postfix(reward=f"{avg_reward:.2f}", loss=f"{avg_loss:.4f}", eps=f"{avg_epsilon:.3f}")

    def _train_agent_locally(self, agent: RLAgent, env: PairedEnv, episodes: int):
        """單個客戶端的本地訓練邏輯。"""
        agent.model.train()
        total_loss, total_reward, episode_count = 0.0, 0.0, 0
        if episodes == 0: return 0.0, 0.0, 0.0

        for ep in range(episodes):
            state, episode_reward, done = env.reset(), 0.0, False
            for step in range(self.config.steps_per_episode):
                action = agent.act(state)
                next_state, reward, done, _ = env.step(action)
                agent.remember(state, action, reward, next_state, done)
                state = next_state
                episode_reward += reward

                if len(agent.memory) > self.config.replay_start_size and step % self.config.replay_frequency == 0:
                    loss = agent.replay(num_batches=self.config.replay_batches_per_call)
                    total_loss += loss

                if done: break

            total_reward += episode_reward
            episode_count += 1
            if (ep + 1) % self.config.target_update_freq == 0:
                agent.update_target_model()

        # 更新 Epsilon
        if not agent.is_eval_agent and agent.epsilon > self.config.epsilon_min:
            agent.epsilon *= self.config.epsilon_decay ** episodes

        avg_loss = total_loss / (episodes * self.config.steps_per_episode / self.config.replay_frequency) if total_loss > 0 else 0
        avg_reward = total_reward / episode_count if episode_count > 0 else 0
        privacy_cost = agent.get_privacy_cost()
        return avg_loss, avg_reward, privacy_cost

    def _run_final_evaluation(self):
        """在所有客戶端上運行最終評估。"""
        print("\n[評估] 正在執行最終評估...")
        for cid, env in tqdm(self.client_envs.items(), desc="最終評估"):
            eval_row = {'client_id': cid}

            base_model_state = self.global_model_state
            # 在 ClusteredFL 中，個性化模型是聚類模型
            if self.config.mode == 'ClusteredFL':
                cluster_id = self.server.client_to_cluster.get(cid, 0)
                personalized_model_state = self.server.cluster_models.get(cluster_id, base_model_state)
            else:
                personalized_model_state = base_model_state

            eval_row['reward_global'] = self._evaluate_on_env(env, base_model_state)
            eval_row['reward_personalized'] = self._evaluate_on_env(env, personalized_model_state)

            if self.config.use_pfl_finetune:
                eval_row['reward_pfl_finetuned'] = self._finetune_and_evaluate(env, personalized_model_state)
            else:
                eval_row['reward_pfl_finetuned'] = eval_row['reward_personalized']

            self.evaluation_results.append(eval_row)

    def _evaluate_on_env(self, env: PairedEnv, model_state: dict, num_episodes: int = 15) -> float:
        """在給定環境上評估特定模型。"""
        if env.trajectory.size == 0 or not model_state: return 0.0

        eval_config = copy.deepcopy(self.config); eval_config.enable_dp = False
        eval_agent = RLAgent(env.state_size, env.action_size, eval_config, -1, 1, True)

        model_to_load = {k: v.to(eval_agent.device).float() for k, v in model_state.items()}
        eval_agent.model.load_state_dict(model_to_load)
        eval_agent.model.eval()
        eval_agent.epsilon = 0.0

        total_reward = 0
        for _ in range(num_episodes):
            state, ep_reward, done = env.reset(), 0, False
            for _ in range(self.config.steps_per_episode):
                action = eval_agent.act(state)
                next_state, reward, done, _ = env.step(action)
                ep_reward += reward
                state = next_state
                if done: break
            total_reward += ep_reward
        return total_reward / num_episodes

    def _finetune_and_evaluate(self, env: PairedEnv, model_state: dict) -> float:
        """對模型進行個性化微調並評估。"""
        if env.trajectory.size == 0 or not model_state: return 0.0

        finetune_config = copy.deepcopy(self.config); finetune_config.enable_dp = False
        finetune_agent = RLAgent(env.state_size, env.action_size, finetune_config, -1, 1, False)

        model_to_load = {k: v.to(finetune_agent.device).float() for k, v in model_state.items()}
        finetune_agent.model.load_state_dict(model_to_load)
        finetune_agent.epsilon = 0.1 # 微調時使用少量探索

        self._train_agent_locally(finetune_agent, env, episodes=self.config.local_finetune_episodes)

        return self._evaluate_on_env(env, finetune_agent.get_clean_state_dict())

    def run(self):
        """啟動並運行整個實驗流程。"""
        print(f"\n{'='*20} 🏃‍♂️ 開始執行實驗: {self.config.experiment_name} ({self.config.mode}) {'='*20}")
        start_time = time.time()

        # 根據模式選擇訓練流程
        # 目前所有模式都走 federated 流程，Centralized/Isolated 可作為未來擴展
        if self.config.mode in ['FedAvg', 'FedProx', 'ClusteredFL', 'CQL']:
             self._run_federated_training()
        else:
            raise ValueError(f"未知的實驗模式: {self.config.mode}")

        self._run_final_evaluation()
        total_time_minutes = (time.time() - start_time) / 60
        print(f"\n✅ 實驗 {self.config.experiment_name} 完成！總耗時: {total_time_minutes:.2f} 分鐘")

        # --- 最終報告 ---
        # (任務 5) 延遲報告
        if self.latency_logs:
            avg_latency = np.mean(self.latency_logs)
            p95_latency = np.percentile(self.latency_logs, 95)
            print(f"⏱️  通信輪延遲報告: 平均 = {avg_latency:.3f} 秒, 95百分位 = {p95_latency:.3f} 秒")

        # (任務 2) 隱私與獎勵報告
        if self.training_history and self.privacy_costs:
            history_df = pd.DataFrame(self.training_history)
            privacy_df = pd.DataFrame(self.privacy_costs)
            privacy_df['cumulative_epsilon'] = privacy_df['epsilon'].cumsum()

            reward_vs_epsilon_df = pd.merge(history_df, privacy_df, on='round')
            reward_vs_epsilon_path = os.path.join(self.config.output_dir, f'{self.config.experiment_name}_reward_vs_epsilon.csv')
            reward_vs_epsilon_df.to_csv(reward_vs_epsilon_path, index=False)
            print(f"💾 獎勵 vs. Epsilon 曲線數據已保存至: {reward_vs_epsilon_path}")

            final_epsilon = reward_vs_epsilon_df['cumulative_epsilon'].iloc[-1]
            print(f"🛡️  最終總隱私預算消耗: ε = {final_epsilon:.4f}")

        # 保存其他結果
        pd.DataFrame(self.evaluation_results).to_csv(os.path.join(self.config.output_dir, f'{self.config.experiment_name}_evaluation_results.csv'), index=False)

        return pd.DataFrame(self.evaluation_results), pd.DataFrame(self.training_history)


print("✅ Cell 7: ExperimentRunner（專家修正版）定義完成。")

✅ Cell 7: ExperimentRunner（專家修正版）定義完成。


In [9]:
# @title Cell 8A: 🎬 實驗執行主函數（專家修正版）
import time
import gc
from datetime import datetime

def run_single_experiment(config_dict: dict, data_path: str):
    """
    運行單次端到端的聯邦學習實驗。

    Args:
        config_dict (dict): 包含實驗參數的字典。
        data_path (str): 數據文件的路徑。

    Returns:
        一個元組 (success, results)，其中 results 是一個包含關鍵指標的字典。
    """
    start_time = time.time()
    runner = None
    results = {}

    try:
        # 1. 初始化配置和數據管理器
        config = TrainingConfig(**config_dict)
        print(f"\n{'='*20} 正在啟動實驗: {config.experiment_name} {'='*20}")
        data_manager = DataManager(data_path, config)

        # 2. 創建實驗執行器 (內部會自動處理數據切分和代理創建)
        runner = ExperimentRunner(config, data_manager)

        # 3. 運行實驗
        eval_res, _ = runner.run()

        # 4. 收集結果
        execution_time = (time.time() - start_time) / 60
        results['execution_time'] = execution_time
        print(f"\n⏱️ 實驗執行時間: {execution_time:.2f} 分鐘")

        if not eval_res.empty:
            avg_rewards = {
                'global': eval_res['reward_global'].mean(),
                'personalized': eval_res['reward_personalized'].mean(),
                'pfl': eval_res['reward_pfl_finetuned'].mean()
            }
            results['avg_rewards'] = avg_rewards
            print("\n✅ 實驗結果摘要:")
            print(f"   - 平均全局獎勵: {avg_rewards['global']:.4f}")
            print(f"   - 平均個性化獎勵: {avg_rewards['personalized']:.4f}")
            print(f"   - 平均PFL微調獎勵: {avg_rewards['pfl']:.4f}")

        # 收集隱私成本
        privacy_file = os.path.join(config.output_dir, f'{config.experiment_name}_reward_vs_epsilon.csv')
        if os.path.exists(privacy_file):
            privacy_df = pd.read_csv(privacy_file)
            final_epsilon = privacy_df['cumulative_epsilon'].iloc[-1]
            results['privacy_stats'] = {'consumed_epsilon': final_epsilon}
            print(f"   - 最終隱私消耗 ε: {final_epsilon:.4f}")

        return True, results

    except Exception as e:
        execution_time = (time.time() - start_time) / 60
        print(f"\n❌ 實驗 '{config_dict.get('experiment_name')}' 失敗！ (耗時: {execution_time:.2f} 分鐘)")
        import traceback
        print(f"🔍 錯誤詳情: {e}")
        print(f"📋 錯誤堆疊: {traceback.format_exc()}")
        return False, {'error': str(e)}

    finally:
        # 清理資源
        del runner
        torch.cuda.empty_cache()
        gc.collect()

print("✅ Cell 8A: 實驗執行主函數（專家修正版）已載入。")

✅ Cell 8A: 實驗執行主函數（專家修正版）已載入。


In [10]:
# @title Cell 8B: 🔧 環境設定與初始化（確定性修正版）
import os
import time

# --- 【錯誤修正】 ---
# (任務 6 的補強) 根據 PyTorch 錯誤提示，設定此環境變數以啟用 CuBLAS 的確定性演算法。
# 這必須在任何 PyTorch/CUDA 操作之前完成。
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'
# --- END 修正 ---

# GPU環境設定
setup_gpu_environment()

# 環境路徑設定
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    BASE_WORK_DIR = "/content/drive/MyDrive/FRL_Slicing_Sim" # 使用新目錄
    print("🔗 Google Drive 掛載成功")
except ImportError:
    BASE_WORK_DIR = "./FRL_Slicing_Sim"
    print("💻 本地環境模式")

# 確保路徑存在
os.makedirs(BASE_WORK_DIR, exist_ok=True)
# 注意: 確保您的 kpi_traces_final_robust0.parquet 文件位於 BASE_WORK_DIR 中
DATA_PATH = os.path.join(BASE_WORK_DIR, "kpi_traces_final_robust0.parquet")
BASE_OUTPUT_DIR = os.path.join(BASE_WORK_DIR, "outputs_expert")
os.makedirs(BASE_OUTPUT_DIR, exist_ok=True)

if not os.path.exists(DATA_PATH):
    print(f"❌ 錯誤: 找不到數據文件！請將 'kpi_traces_final_robust0.parquet' 上傳到 '{BASE_WORK_DIR}'")
else:
    print(f"📁 數據路徑: {DATA_PATH}")

print(f"📁 輸出目錄: {BASE_OUTPUT_DIR}")

# 實驗配置
# 新增 CQL 作為對比
MODES_TO_RUN = ["ClusteredFL", "FedProx", "FedAvg", "CQL"]
SEEDS = [42] # 為節省時間，僅用一個種子

print(f"\n🎯 實驗計劃:")
print(f"   - 測試模式: {MODES_TO_RUN}")
print(f"   - 隨機種子: {SEEDS}")
print(f"   - 總實驗數: {len(MODES_TO_RUN) * len(SEEDS)}")

# 全局結果存儲
master_results_log = []

print("\n✅ 環境設定完成，準備執行實驗。")


🎮 GPU 檢測: Tesla T4
🧹 GPU 環境已設定並優化。
Mounted at /content/drive
🔗 Google Drive 掛載成功
📁 數據路徑: /content/drive/MyDrive/FRL_Slicing_Sim/kpi_traces_final_robust0.parquet
📁 輸出目錄: /content/drive/MyDrive/FRL_Slicing_Sim/outputs_expert

🎯 實驗計劃:
   - 測試模式: ['ClusteredFL', 'FedProx', 'FedAvg', 'CQL']
   - 隨機種子: [42]
   - 總實驗數: 4

✅ 環境設定完成，準備執行實驗。


In [ ]:
# @title Cell 8C: 🚀 實驗 1 - ClusteredFL（專家修正版）
# 確保環境已準備就緒
if 'BASE_OUTPUT_DIR' not in locals():
    print("請先運行 Cell 8B 進行環境設定！")
else:
    mode = "ClusteredFL"
    for seed in SEEDS:
        exp_name = f"{mode}_expert_s{seed}"
        output_dir = os.path.join(BASE_OUTPUT_DIR, f"seed_{seed}", mode)

        # ClusteredFL 專用配置
        config = {
            "experiment_name": exp_name,
            "output_dir": output_dir,
            "mode": mode,
            "random_seed": seed,
            "comm_rounds": 5, # 示例訓練輪數

            "num_clients": 15,
            "dirichlet_alpha": 0.4,
            "num_clients_to_select": 8,

            "local_episodes_per_round": 4,
            "batch_size": 128,

            # 模式特定參數
            "fedprox_mu": 0.1, # 在 ClusteredFL 中也可用於懲罰與聚類中心的距離
            "num_clusters": 3,
            "cluster_update_freq": 3,

            "enable_dp": True,
            "dp_target_epsilon": 8.0,
            "dp_noise_multiplier": 0.6,
            "dp_max_grad_norm": 1.5, # 假設一個初始值，如果啟用自適應會被覆蓋
            "enable_adaptive_clipping": True,
            "adaptive_clipping_percentile": 0.8,

            "device": "cuda" if torch.cuda.is_available() else "cpu"
        }

        # 執行實驗
        success, result_info = run_single_experiment(config, DATA_PATH)

        # 記錄結果
        master_results_log.append({
            'seed': seed, 'mode': mode, 'success': success, 'result_info': result_info
        })

🚀 GPU 環境檢測到: Tesla T4

--- 核心實驗配置 ---
模式: ClusteredFL | 客戶端數: 15 (Non-IID, α=0.4)
每輪參與: 8 | 通信輪數: 5
🛡️  差分隱私 (DP): 啟用
   - 目標預算: ε=8.0, δ=1e-05
   - 噪聲乘數: 0.6
   - 梯度裁剪: 自適應
   - 預算重設: 啟用

==================== 正在啟動實驗: ClusteredFL_expert_s42 ====================

[DataManager] 正在從 /content/drive/MyDrive/FRL_Slicing_Sim/kpi_traces_final_robust0.parquet 讀取數據...

==================== DataManager 啟動前預檢查 ====================
✅ 清理後的欄位列表 (共 38 個)
   - 吞吐量欄位成功匹配: 'throughput_dl_mbps'
   - 延遲/緩衝區欄位成功匹配: 'buffer_occupancy_dl_bytes'

torch.use_deterministic_algorithms(True) 已設置

[DataManager] 正在為 15 個客戶端生成 Non-IID 數據分區...
   - Dirichlet Alpha (α): 0.4
[DataManager] 正在合併 eMBB 和 URLLC 的數據軌跡...
[DataManager] 全局數據軌跡合併完成，共 196183 個時間步。


創建客戶端軌跡:   0%|          | 0/15 [00:00<?, ?it/s]

   - 客戶端 0: 3180 個時間步
   - 客戶端 1: 18666 個時間步
   - 客戶端 2: 500 個時間步
   - 客戶端 3: 500 個時間步
   - 客戶端 4: 10379 個時間步
   - 客戶端 5: 500 個時間步
   - 客戶端 6: 32423 個時間步
   - 客戶端 7: 522 個時間步
   - 客戶端 8: 1891 個時間步
   - 客戶端 9: 4542 個時間步
   - 客戶端 10: 1709 個時間步
   - 客戶端 11: 5204 個時間步
   - 客戶端 12: 660 個時間步
   - 客戶端 13: 500 個時間步
   - 客戶端 14: 115007 個時間步

[DataManager] Non-IID 數據處理完成！成功為 15 / 15 個客戶端創建了環境。

[Adaptive Clipper] 正在預計算梯度裁剪範數...
   - 執行預熱步驟以收集梯度範數...
   - ✅ 預計算完成。建議的裁剪範數 (80 百分位): 1661435.57
[FLServer] 初始化完成 - 模式: ClusteredFL, 聚類數: 3

[ExperimentRunner] 正在初始化所有客戶端代理...
[C-0] 🛡️ 正在初始化差分隱私引擎...
   - ✅ 差分隱私引擎初始化成功 (GradSampleModule)
[C-1] 🛡️ 正在初始化差分隱私引擎...
   - ✅ 差分隱私引擎初始化成功 (GradSampleModule)
[C-2] 🛡️ 正在初始化差分隱私引擎...
   - ✅ 差分隱私引擎初始化成功 (GradSampleModule)
[C-3] 🛡️ 正在初始化差分隱私引擎...
   - ✅ 差分隱私引擎初始化成功 (GradSampleModule)
[C-4] 🛡️ 正在初始化差分隱私引擎...
   - ✅ 差分隱私引擎初始化成功 (GradSampleModule)
[C-5] 🛡️ 正在初始化差分隱私引擎...
   - ✅ 差分隱私引擎初始化成功 (GradSampleModule)
[C-6] 🛡️ 正在初始化差分隱私引擎...
   - ✅ 差分隱私引擎初始化成功 (GradSampleModule)
[

ClusteredFL Training:   0%|          | 0/5 [00:00<?, ?it/s]

[Round 1] ⚠️ 近即時延遲警告: 本輪耗時 1022.09 秒 > 1 秒。
[Round 2] ⚠️ 近即時延遲警告: 本輪耗時 1984.96 秒 > 1 秒。
[Round 3] ⚠️ 近即時延遲警告: 本輪耗時 2238.01 秒 > 1 秒。

[FLServer] Round 3: 正在更新客戶端聚類...
[FLServer] 聚類更新完成:
  - 聚類 0: [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 14]
  - 聚類 1: [13]
  - 聚類 2: [6]


In [ ]:
# @title Cell 8D: 🚀 實驗 2 - FedProx & FedAvg（專家修正版）
if 'BASE_OUTPUT_DIR' not in locals():
    print("請先運行 Cell 8B 進行環境設定！")
else:
    for mode in ["FedProx", "FedAvg"]:
        for seed in SEEDS:
            exp_name = f"{mode}_expert_s{seed}"
            output_dir = os.path.join(BASE_OUTPUT_DIR, f"seed_{seed}", mode)

            config = {
                "experiment_name": exp_name,
                "output_dir": output_dir,
                "mode": mode,
                "random_seed": seed,
                "comm_rounds": 5,

                "num_clients": 15,
                "dirichlet_alpha": 0.4,
                "num_clients_to_select": 8,

                "local_episodes_per_round": 4,
                "batch_size": 128,

                # FedProx 的 mu 值，在 FedAvg 模式下會被自動設為 0
                "fedprox_mu": 0.1,

                "enable_dp": True,
                "dp_target_epsilon": 8.0,
                "dp_noise_multiplier": 0.6,
                "dp_max_grad_norm": 1.5,
                "enable_adaptive_clipping": True,
                "adaptive_clipping_percentile": 0.8,

                "device": "cuda" if torch.cuda.is_available() else "cpu"
            }

            # 執行實驗
            success, result_info = run_single_experiment(config, DATA_PATH)

            # 記錄結果
            master_results_log.append({
                'seed': seed, 'mode': mode, 'success': success, 'result_info': result_info
            })

In [ ]:
# @title Cell 8E: 🚀 實驗 3 - CQL Baseline（專家修正版）
if 'BASE_OUTPUT_DIR' not in locals():
    print("請先運行 Cell 8B 進行環境設定！")
else:
    mode = "CQL"
    for seed in SEEDS:
        exp_name = f"{mode}_expert_s{seed}"
        output_dir = os.path.join(BASE_OUTPUT_DIR, f"seed_{seed}", mode)

        config = {
            "experiment_name": exp_name,
            "output_dir": output_dir,
            "mode": mode,
            "random_seed": seed,
            "comm_rounds": 5,

            "num_clients": 15,
            "dirichlet_alpha": 0.4,
            "num_clients_to_select": 8,

            "local_episodes_per_round": 4,
            "batch_size": 128,

            # CQL 特定參數
            "enable_cql": True,
            "cql_alpha": 7.5,
            "cql_temperature": 1.0,

            # CQL 是一種非聯邦的聚合方法，但我們這裡模擬在 FL 設定下的表現
            # 因此 fedprox_mu 設為 0
            "fedprox_mu": 0.0,

            "enable_dp": True,
            "dp_target_epsilon": 8.0,
            "dp_noise_multiplier": 0.6,
            "dp_max_grad_norm": 1.5,
            "enable_adaptive_clipping": True,
            "adaptive_clipping_percentile": 0.8,

            "device": "cuda" if torch.cuda.is_available() else "cpu"
        }

        # 執行實驗
        success, result_info = run_single_experiment(config, DATA_PATH)

        # 記錄結果
        master_results_log.append({
            'seed': seed, 'mode': mode, 'success': success, 'result_info': result_info
        })

In [ ]:
# @title Cell 8F: 📊 實驗總結（專家修正版）
print("="*60)
print("🎉 所有實驗執行完成！")
print("="*60)

# 統計成功率
successful_experiments = sum(1 for r in master_results_log if r['success'])
total_experiments = len(master_results_log)
success_rate = (successful_experiments / total_experiments) * 100 if total_experiments > 0 else 0

print(f"\n📊 實驗統計:")
print(f"   - 總實驗數: {total_experiments}")
print(f"   - 成功數: {successful_experiments}")
print(f"   - 成功率: {success_rate:.1f}%\n")

# 成功實驗的詳細統計
summary_data = []
for result in master_results_log:
    if result['success']:
        mode = result['mode']
        info = result['result_info']
        if info and 'avg_rewards' in info:
            rewards = info['avg_rewards']
            privacy = info.get('privacy_stats', {'consumed_epsilon': 0})

            summary_data.append({
                'Mode': mode,
                'Global Reward': rewards.get('global', 0),
                'Personalized Reward': rewards.get('personalized', 0),
                'PFL Reward': rewards.get('pfl', 0),
                'Epsilon': privacy.get('consumed_epsilon', 0),
                'Time (min)': info.get('execution_time', 0)
            })

if summary_data:
    summary_df = pd.DataFrame(summary_data)
    summary_df = summary_df.round(4)

    # 計算提升率和隱私效率
    best_global_reward = summary_df.loc[summary_df['Mode'] != 'CQL', 'Global Reward'].max()
    summary_df['PFL Improvement (%)'] = (summary_df['PFL Reward'] / summary_df['Global Reward'] - 1) * 100
    summary_df['Privacy-Utility'] = summary_df['PFL Reward'] / summary_df['Epsilon'].replace(0, 1e-9)

    summary_df = summary_df.round(2)

    print("📈 成功實驗詳細結果:")
    print(summary_df.to_markdown(index=False))

    # 比較不同方法
    if len(summary_df) > 1:
        best_pfl = summary_df.loc[summary_df['PFL Reward'].idxmax()]
        best_privacy_utility = summary_df.loc[summary_df['Privacy-Utility'].idxmax()]

        print(f"\n🏆 最佳表現:")
        print(f"   - 最佳 PFL 獎勵: {best_pfl['Mode']} ({best_pfl['PFL Reward']:.4f})")
        print(f"   - 最佳隱私效用權衡: {best_privacy_utility['Mode']} ({best_privacy_utility['Privacy-Utility']:.4f} Reward/ε)")

# 保存實驗總結
summary_path = os.path.join(BASE_OUTPUT_DIR, "experiment_summary.json")
with open(summary_path, 'w') as f:
    # 創建一個可序列化的版本
    serializable_log = []
    for log in master_results_log:
        new_log = log.copy()
        if isinstance(new_log.get('result_info'), dict):
             new_log['result_info'].pop('error', None) # 移除不可序列化的 Error 對象
        serializable_log.append(new_log)
    json.dump(serializable_log, f, indent=4)
print(f"\n📄 實驗總結已保存至: {summary_path}")

In [ ]:
# @title Cell 9: 📊 結果視覺化（專家修正版）
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob
import json
import numpy as np

def load_all_expert_results(base_output_dir):
    """
    從指定的輸出目錄加載所有專家實驗的 reward_vs_epsilon.csv 和 evaluation_results.csv 文件。
    """
    all_evals, all_reward_vs_eps = [], []

    if not os.path.exists(base_output_dir):
        print(f"❌ 結果目錄未找到: {base_output_dir}")
        return pd.DataFrame(), pd.DataFrame()

    eval_files = glob.glob(os.path.join(base_output_dir, '**', '*_evaluation_results.csv'), recursive=True)
    reward_eps_files = glob.glob(os.path.join(base_output_dir, '**', '*_reward_vs_epsilon.csv'), recursive=True)

    def extract_info_from_path(path):
        parts = path.split(os.sep)
        # 預期路徑結構: .../seed_XX/MODE/...
        mode = parts[-2]
        seed_str = [p for p in parts if p.startswith('seed_')][0]
        seed = int(seed_str.split('_')[1])
        return mode, seed

    for f in eval_files:
        try:
            df = pd.read_csv(f)
            mode, seed = extract_info_from_path(f)
            df['mode'] = mode
            df['seed'] = seed
            all_evals.append(df)
        except Exception as e:
            print(f"🟡 警告: 讀取評估文件失敗: {f}, {e}")

    for f in reward_eps_files:
        try:
            df = pd.read_csv(f)
            mode, seed = extract_info_from_path(f)
            df['mode'] = mode
            df['seed'] = seed
            all_reward_vs_eps.append(df)
        except Exception as e:
            print(f"🟡 警告: 讀取獎勵文件失敗: {f}, {e}")

    return (pd.concat(all_evals, ignore_index=True) if all_evals else pd.DataFrame()), \
           (pd.concat(all_reward_vs_eps, ignore_index=True) if all_reward_vs_eps else pd.DataFrame())

# --- 視覺化設定 ---
sns.set_theme(style="whitegrid", palette="muted", font_scale=1.2)
FIGURES_OUTPUT_DIR = os.path.join(BASE_OUTPUT_DIR, "figures")
os.makedirs(FIGURES_OUTPUT_DIR, exist_ok=True)

print(f"🔍 正在從以下路徑加載結果: {BASE_OUTPUT_DIR}")
eval_df, reward_vs_eps_df = load_all_expert_results(BASE_OUTPUT_DIR)

if eval_df.empty or reward_vs_eps_df.empty:
    print("❌ 未找到任何結果文件，無法生成圖表。請確保 Cell 8 的實驗已成功完成。")
else:
    mode_order = ["FedAvg", "FedProx", "ClusteredFL", "CQL"]
    print(f"✅ 成功加載了 {len(eval_df['seed'].unique())} 次運行的結果。")
    print(f"📊 找到的模式: {sorted(eval_df['mode'].unique())}")

    # --- 圖 1: 訓練歷史（獎勵 vs. 通信輪） ---
    plt.figure(figsize=(12, 7))
    sns.lineplot(data=reward_vs_eps_df, x='round', y='avg_reward', hue='mode',
                 hue_order=[m for m in mode_order if m in reward_vs_eps_df['mode'].unique()],
                 errorbar='sd', linewidth=2.5)
    plt.title('Training Performance Comparison', fontsize=18, weight='bold')
    plt.xlabel('Communication Round', fontsize=14)
    plt.ylabel('Average Episodic Reward', fontsize=14)
    plt.legend(title='Training Mode', fontsize=12)
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.tight_layout()
    plt.savefig(os.path.join(FIGURES_OUTPUT_DIR, 'expert_training_history.png'), dpi=300)
    plt.show()

    # --- 圖 2: 最終性能比較 (PFL Reward) ---
    plt.figure(figsize=(12, 7))
    sns.boxplot(data=eval_df, x='mode', y='reward_pfl_finetuned',
                order=[m for m in mode_order if m in eval_df['mode'].unique()])
    plt.title('Final Performance After PFL Fine-tuning', fontsize=18, weight='bold')
    plt.xlabel('Experiment Mode', fontsize=14)
    plt.ylabel('Final Reward Score', fontsize=14)
    plt.xticks(rotation=10)
    plt.tight_layout()
    plt.savefig(os.path.join(FIGURES_OUTPUT_DIR, 'expert_final_performance.png'), dpi=300)
    plt.show()

    # --- 圖 3: 隱私-效用權衡曲線 ---
    plt.figure(figsize=(12, 7))
    sns.lineplot(data=reward_vs_eps_df, x='cumulative_epsilon', y='avg_reward', hue='mode',
                 hue_order=[m for m in mode_order if m in reward_vs_eps_df['mode'].unique()],
                 errorbar='sd', linewidth=2.5, marker='o', markersize=5, markevery=1)
    plt.title('Privacy-Utility Trade-off Curve', fontsize=18, weight='bold')
    plt.xlabel('Cumulative Privacy Loss ε (Epsilon)', fontsize=14)
    plt.ylabel('Average Episodic Reward', fontsize=14)
    plt.legend(title='Training Mode')
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.tight_layout()
    plt.savefig(os.path.join(FIGURES_OUTPUT_DIR, 'expert_privacy_utility_tradeoff.png'), dpi=300)
    plt.show()

    # --- 圖 4: 個性化效益分析 (以 ClusteredFL 為例) ---
    if 'ClusteredFL' in eval_df['mode'].unique():
        target_eval = eval_df[eval_df['mode'] == 'ClusteredFL']
        target_melted = target_eval.melt(
            id_vars=['client_id'],
            value_vars=['reward_global', 'reward_personalized', 'reward_pfl_finetuned'],
            var_name='Model Type', value_name='Average Reward'
        ).replace({
            'reward_global': 'Global', 'reward_personalized': 'Clustered', 'reward_pfl_finetuned': 'PFL-Finetuned'
        })

        plt.figure(figsize=(15, 7))
        sns.barplot(data=target_melted, x='client_id', y='Average Reward', hue='Model Type', palette='viridis')
        plt.title('Personalization Benefits in ClusteredFL', fontsize=16, weight='bold')
        plt.xlabel('Client ID', fontsize=14)
        plt.ylabel('Average Reward', fontsize=14)
        plt.tight_layout()
        plt.savefig(os.path.join(FIGURES_OUTPUT_DIR, 'expert_personalization_benefit.png'), dpi=300)
        plt.show()

print(f"✅ Cell 9: 結果視覺化（專家修正版）已完成。")
print(f"📁 所有圖表已保存至: {FIGURES_OUTPUT_DIR}")